### Import the necessary library

In [1]:
import torch.utils.data
import scanpy as sc
from sklearn.cluster import KMeans
from scipy.spatial import distance
import scipy.sparse
import scib 
import numpy as np
import random
import pandas as pd
# from torch.distribctions import  kl_divergence as kl
# torch.backends.cudnn.benchmark = True
from sklearn.neighbors import KDTree
from sklearn import preprocessing
import scipy
import scib

In [2]:
import scanpy as sc
import numpy as np
import pandas as pd
import time
import scvi
import anndata
import pandas as pd
from scipy.io import mmread
from scipy.sparse import csr_matrix
import umap
import matplotlib.pyplot as plt
import os
import tempfile
import seaborn as sns
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings("ignore")

/opt/anaconda3/lib/python3.12/site-packages/lightning/fabric/__init__.py:40: Deprecated call to `pkg_resources.declare_namespace('lightning.fabric')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
/opt/anaconda3/lib/python3.12/site-packages/pkg_resources/__init__.py:2317: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('lightning')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(parent)
/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/__init__.py:37: Deprecated call to `pkg_resources.declare_namespace('lightning.pytorch')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to 

In [3]:
import warnings

sc.set_figure_params(figsize=(6, 6), frameon=False)
sns.set_theme()
torch.set_float32_matmul_precision("high")
save_dir = tempfile.TemporaryDirectory()

%config InlineBackend.print_figure_kwargs={"facecolor": "w"}
%config InlineBackend.figure_format="retina"

import warnings
warnings.filterwarnings("ignore")

# Suppress specific ImportWarning
warnings.filterwarnings("ignore", category=ImportWarning, message=".*AltairImportHook.find_spec() not found; falling back to find_module.*")

In [4]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()

#Perhaps require the library relocation (to the library contains kBET and lisi library)
robjects.r('.libPaths(c("current path", "library location"))')

rscript = '''
library(kBET)
# library(lisi)
'''
robjects.r(rscript)
kbet = robjects.r('kBET')
# lisi = robjects.r['compute_lisi']

### read the data

In [5]:
# load the prediction results to adata
import anndata
adata = anndata.read_h5ad("../bct_raw.h5ad")
adata

AnnData object with n_obs × n_vars = 9288 × 1222
    obs: 'orig.ident', 'nCount_originalexp', 'nFeature_originalexp', 'study', 'cell.class', 'library_size', 'detected_genes', 'batch', 'cell_type'
    var: 'gene.symbols', 'gene.counts'

In [6]:
adata

AnnData object with n_obs × n_vars = 9288 × 1222
    obs: 'orig.ident', 'nCount_originalexp', 'nFeature_originalexp', 'study', 'cell.class', 'library_size', 'detected_genes', 'batch', 'cell_type'
    var: 'gene.symbols', 'gene.counts'

In [7]:
# preprocessing
import scanpy as sc
adata.raw = adata
adata.layers["counts"] = adata.X.copy()
sc.pp.filter_cells(adata, min_genes=300)
sc.pp.filter_genes(adata, min_cells=5)
sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=2000, batch_key='batch')
adata = adata[:, adata.var['highly_variable']]

## Print the normal metrics (for scib)

In [6]:
import pandas as pd
import scib

def calculate_metrics_for_embeddings(file_path, adata, embedding_keys, batch_key='batch', label_key='cell_type'):
    # Read the CSV file into a DataFrame
    combined_embeddings = pd.read_csv(file_path, index_col=0)

    # Ensure indices match
    combined_embeddings = combined_embeddings.loc[adata.obs_names]

    # Split the DataFrame into separate DataFrames for each embedding
    embeddings_dict = {key: combined_embeddings.filter(like=key) for key in embedding_keys}

    # Assign the embeddings back to adata.obsm
    for key, df in embeddings_dict.items():
        adata.obsm[key] = df.values

    # Initialize an empty list to store results
    results = []

    # Loop through each embedding and calculate metrics
    for key in embedding_keys:
        print(f"File: {file_path}, Embedding: {key}")
        sc.pp.neighbors(adata, use_rep=key)
        scib.me.cluster_optimal_resolution(adata, cluster_key="cluster", label_key=label_key)
        metrics = scib.me.metrics(
            adata, adata_int=adata, 
            silhouette_=True, graph_conn_=True, 
            ari_=True, nmi_=True, isolated_labels_=True, 
            isolated_labels_f1_=True,
            isolated_labels_asw_=True, 
            batch_key=batch_key, label_key=label_key, 
            embed=key
        )
        
        for metric, score in metrics.items():
            # Store results with identifying information
            result_entry = {
                'file_path': file_path,
                'embedding_key': key,
                'metric': metric,
                'score': score
            }
            results.append(result_entry)
            
            # Print the file path, embedding key, and result
            print(f"File: {file_path}, Embedding: {key}, Metric: {metric}, Score: {score}")

    return results

In [7]:
file_paths = [
    # "../embeddings/full_annotated_supervised_bct_new.csv",
    # "../embeddings/randomly_missing_bct_new_30.csv",
    # "../embeddings/randomly_missing_bct_new_50.csv",
    # "../embeddings/randomly_missing_bct_new_70.csv"
    "../embeddings/full_annotated_unsupervised_bct.csv"
]

# Initialize the list of embedding keys to use
# embedding_keys = ["X_scDREAMER", "X_ItClust"]
embedding_keys = ["X_scanorama"]

# Prepare a list to collect all results
all_results = []

# Loop through each file and calculate metrics
for file_path in file_paths:
    results = calculate_metrics_for_embeddings(file_path, adata, embedding_keys)
    all_results.extend(results)

# Now `all_results` contains metrics for all embedding files with identifying information

/var/folders/nx/dqg67h8n5t1fhb63d87vq4z80000gn/T/ipykernel_22296/3871537999.py:16: ImplicitModificationWarning: Setting element `.obsm['X_scanorama']` of view, initializing view as actual.
  adata.obsm[key] = df.values


File: ../embeddings/full_annotated_unsupervised_bct.csv, Embedding: X_scanorama


/Users/sheena/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/sheena/Library/Python/3.9/lib/python/site-packages/scib/metrics/clustering.py:101: FutureWarning: In the future, the default backend for leiden will be igraph instead of leidenalg.

 To achieve the future defaults please pass: flavor="igraph" and n_iterations=2.  directed must also be False to work with igraph's implementation.
  cluster_function(adata, resolution=res, key_added=cluster_key, **kwargs)


resolution: 0.1, nmi: 0.9636336320181687
resolution: 0.2, nmi: 0.834771459257282
resolution: 0.3, nmi: 0.8177281239589398
resolution: 0.4, nmi: 0.8185444579217254
resolution: 0.5, nmi: 0.8184792922770202
resolution: 0.6, nmi: 0.7268206578601852
resolution: 0.7, nmi: 0.7264704718299608
resolution: 0.8, nmi: 0.7129711887340887
resolution: 0.9, nmi: 0.6881578366091975
resolution: 1.0, nmi: 0.6556247748920834
resolution: 1.1, nmi: 0.6536286538264633
resolution: 1.2, nmi: 0.6496694901120496
resolution: 1.3, nmi: 0.6491211940771829
resolution: 1.4, nmi: 0.6443528230649164
resolution: 1.5, nmi: 0.6433423358488408
resolution: 1.6, nmi: 0.6373493431050469
resolution: 1.7, nmi: 0.641013028995744
resolution: 1.8, nmi: 0.6096627671016959
resolution: 1.9, nmi: 0.5862846745840241
resolution: 2.0, nmi: 0.5551760729622569
optimised clustering against cell_type
optimal cluster resolution: 0.1
optimal score: 0.9636336320181687


/Users/sheena/Library/Python/3.9/lib/python/site-packages/scib/metrics/metrics.py:293: DeprecationWarning: Call to deprecated function (or staticmethod) opt_louvain.
  res_max, nmi_max, nmi_all = opt_louvain(


...

/Users/sheena/Library/Python/3.9/lib/python/site-packages/scib/metrics/isolated_labels.py:272: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  batch_per_lab = tmp.groupby(label_key).agg({batch_key: "count"})


Compute neighbors on rep X_scanorama
Compute neighbors on rep X_scanorama
Isolated labels ASW...


/Users/sheena/Library/Python/3.9/lib/python/site-packages/scib/metrics/isolated_labels.py:272: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  batch_per_lab = tmp.groupby(label_key).agg({batch_key: "count"})


Graph connectivity...
File: ../embeddings/full_annotated_unsupervised_bct.csv, Embedding: X_scanorama, Metric: 0, Score: NMI_cluster/label            0.964939
ARI_cluster/label            0.982933
ASW_label                    0.680228
ASW_label/batch              0.902023
PCR_batch                         NaN
cell_cycle_conservation           NaN
isolated_label_F1            0.991583
isolated_label_silhouette    0.675281
graph_conn                   1.000000
kBET                              NaN
iLISI                             NaN
cLISI                             NaN
hvg_overlap                       NaN
trajectory                        NaN
Name: 0, dtype: float64


/Users/sheena/Library/Python/3.9/lib/python/site-packages/scib/metrics/graph_connectivity.py:56: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  tab = pd.value_counts(labels)
/Users/sheena/Library/Python/3.9/lib/python/site-packages/scib/metrics/graph_connectivity.py:56: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  tab = pd.value_counts(labels)
/Users/sheena/Library/Python/3.9/lib/python/site-packages/scib/metrics/graph_connectivity.py:56: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  tab = pd.value_counts(labels)


In [8]:
# Flatten the nested dictionaries and extract desired scores
results = []
for entry in all_results:
    file_path = entry['file_path']
    embedding_key = entry['embedding_key']
    score_series = entry['score']
    for metric, value in score_series.items():
        # Create an entry for each score
        results.append({
            'file_path': file_path,
            'embedding_key': embedding_key,
            'metric': metric,
            'score': value
        })

# Convert results list to DataFrame
results_df = pd.DataFrame(results)

In [9]:
results_df

,file_path,embedding_key,metric,score
0,../embeddings/full_annotated_unsupervised_bct.csv,X_scanorama,NMI_cluster/label,0.964939
1,../embeddings/full_annotated_unsupervised_bct.csv,X_scanorama,ARI_cluster/label,0.982933
2,../embeddings/full_annotated_unsupervised_bct.csv,X_scanorama,ASW_label,0.680228
3,../embeddings/full_annotated_unsupervised_bct.csv,X_scanorama,ASW_label/batch,0.902023
4,../embeddings/full_annotated_unsupervised_bct.csv,X_scanorama,PCR_batch,NaN
5,../embeddings/full_annotated_unsupervised_bct.csv,X_scanorama,cell_cycle_conservation,NaN
6,../embeddings/full_annotated_unsupervised_bct.csv,X_scanorama,isolated_label_F1,0.991583
7,../embeddings/full_annotated_unsupervised_bct.csv,X_scanorama,isolated_label_silhouette,0.675281
8,../embeddings/full_annotated_unsupervised_bct.csv,X_scanorama,graph_conn,1.000000
9,../embeddings/full_annotated_unsupervised_bct.csv,X_scanorama,kBET,NaN


In [14]:
results_df.to_csv('../metrics/bct_scib_metrics_new.csv', index=True)

## Print the overall metrics for TPT

In [19]:
# Positive and true positive cells defined in iMAP
def positive_true_positive(adata, batch_key='batch', celltype_key='cell_type', use_raw=False,k1=20, k2=100, tp_thr=3., distance='cosine', embed='X_pca'):
    celltype_list = adata.obs[celltype_key]
    batch_list = adata.obs[batch_key]

    temp_c = adata.obs[celltype_key].value_counts()
    temp_b = pd.crosstab(adata.obs[celltype_key], adata.obs[batch_key])
    temp_b_prob = temp_b.divide(temp_b.sum(1), axis=0)
    
    if use_raw:
        if isinstance(adata.X, scipy.sparse.csr.csr_matrix):
            X = adata.X.todense()
        else:
            X = adata.X
    else:
        X = adata.obsm[embed]
    if distance == 'cosine':
        X = preprocessing.normalize(X, axis=1)

    t1 = KDTree(X)

    p_list = []
    tp_list = []

    for cell in range(len(X)):

        # Discriminate positive cells
        neig1 = min(k1, temp_c[celltype_list[cell]])
        NNs = t1.query(X[cell].reshape(1,-1), neig1+1, return_distance=False)[0, 1:]
        c_NN = celltype_list[NNs]
        true_rate = sum(c_NN == celltype_list[cell])/neig1
        if true_rate > 0.5:
            p_list.append(True)
        else:
            p_list.append(False)

        # Discriminate true positive cells
        if p_list[cell] == True:
            neig2 = min(k2, temp_c[celltype_list[cell]])
            NNs = t1.query(X[cell].reshape(1,-1), neig2, return_distance=False)[0]
            NNs_c = celltype_list[NNs]
            NNs_i = NNs_c == celltype_list[cell]
            NNs = NNs[NNs_i] # get local neighbors that are from the same cell type
            neig2 = len(NNs)
            NNs_b = batch_list[NNs]

            max_b = 0
            b_prob = temp_b_prob.loc[celltype_list[cell]]
            for b in set(batch_list):
                if b_prob[b] > 0 and b_prob[b] < 1:
                    p_b = sum(NNs_b == b)
                    stat_b = abs(p_b - neig2*b_prob[b]) / np.sqrt(neig2*b_prob[b]*(1-b_prob[b]))
                    max_b = max(max_b, stat_b)
            if max_b <= tp_thr:
                tp_list.append(True)
            else:
                tp_list.append(False)
        else:
            tp_list.append(False)

    pos_rate = sum(p_list)/len(p_list)
    truepos_rate = sum(tp_list)/len(tp_list)
    return pos_rate, truepos_rate


In [20]:
import pandas as pd
import scanpy as sc

def evaluate_embeddings(adata, file_paths, embedding_keys, batch_key='batch', celltype_key='cell_type', use_raw=False, k1=20, k2=100, tp_thr=3.0, distance='cosine'):
    results = []  # Initialize results as a list
    
    for file_path in file_paths:
        # Read the CSV file into a DataFrame
        combined_embeddings = pd.read_csv(file_path, index_col=0)
        
        # Ensure indices match
        combined_embeddings = combined_embeddings.loc[adata.obs_names]
        
        # Split the DataFrame into separate DataFrames for each embedding
        embeddings_dict = {key: combined_embeddings.filter(like=key) for key in embedding_keys}
        
        # Assign the embeddings back to adata.obsm
        for key, df in embeddings_dict.items():
            adata.obsm[key] = df.values
            
            # Evaluate embedding
            pos_rate, truepos_rate = positive_true_positive(
                adata,
                batch_key=batch_key,
                celltype_key=celltype_key,
                use_raw=use_raw,
                k1=k1,
                k2=k2,
                tp_thr=tp_thr,
                distance=distance,
                embed=key
            )
            
            # Store results with identifying information
            result_entry = {
                'file_path': file_path,
                'embedding_key': key,
                'pos_rate': pos_rate,
                'truepos_rate': truepos_rate
            }
            results.append(result_entry)
            
            # Print the file path, embedding key, and result
            print(f"File Path: {file_path}, Embedding Key: {key}, pos_rate: {pos_rate}, truepos_rate: {truepos_rate}")
    
    # Convert results list to DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df

In [21]:
file_paths = [
    # "../embeddings/full_annotated_supervised_bct_new.csv",
    # "../embeddings/randomly_missing_bct_new_30.csv",
    # "../embeddings/randomly_missing_bct_new_50.csv",
    # "../embeddings/randomly_missing_bct_new_70.csv"
    "../embeddings/full_annotated_unsupervised_bct.csv"
]

# Initialize the list of embedding keys to use
# embedding_keys = ["X_scDREAMER", "X_ItClust"]
embedding_keys = ["X_scanorama"]


results_df = evaluate_embeddings(adata, file_paths, embedding_keys)

# Save the DataFrame to a CSV file
# results_df.to_csv('../metrics/bct_true_positive_true_scores_new.csv', index=False)

File Path: ../embeddings/full_annotated_unsupervised_bct.csv, Embedding Key: X_scanorama, pos_rate: 0.995997911954063, truepos_rate: 0.04628501827040195


In [22]:
results_df

,file_path,embedding_key,pos_rate,truepos_rate
0,../embeddings/full_annotated_unsupervised_bct.csv,X_scanorama,0.995998,0.046285


## Print the overall metrics for kBET

In [13]:
import pandas as pd
import scanpy as sc
import numpy as np
import scib  
import scipy.sparse as sp

# teach all csr_matrix objects that `.A` means “dense array”
sp.csr_matrix.A = property(lambda self: self.toarray())

def evaluate_embeddings(adata, file_paths, embedding_keys, batch_key='batch', celltype_key='cell_type'):
    results = []  # Initialize results as a list

    for file_path in file_paths:
        # Read the CSV file into a DataFrame
        combined_embeddings = pd.read_csv(file_path, index_col=0)

        # Ensure indices match
        combined_embeddings = combined_embeddings.loc[adata.obs_names]

        # Split the DataFrame into separate DataFrames for each embedding
        embeddings_dict = {key: combined_embeddings.filter(like=key) for key in embedding_keys}

        # Assign the embeddings back to adata.obsm
        for key, df in embeddings_dict.items():
            adata.obsm[key] = df.values

            # Calculate kBET score
            kbet_score = scib.metrics.kBET(
                adata,
                batch_key=batch_key,
                label_key=celltype_key,
                type_=None,
                embed=key,
                scaled=True,
                verbose=False,
            )

            # Store results with identifying information
            result_entry = {
                'file_path': file_path,
                'embedding_key': key,
                'kbet_score': kbet_score
            }
            results.append(result_entry)

            # Print the file path, embedding key, and result
            print(f"File Path: {file_path}, Embedding Key: {key}, kBET Score: {kbet_score}")

    # Convert results list to DataFrame
    results_df = pd.DataFrame(results)

    return results_df

In [14]:
# file_paths = [
#     "../embeddings/full_annotated_supervised_bct_new.csv",
#     "../embeddings/partially_annotated_batches_bct_new_30.csv",
#     "../embeddings/partially_annotated_batches_bct_new_50.csv",
#     "../embeddings/partially_annotated_batches_bct_new_70.csv",
#     "../embeddings/mixing_at_edge_bct_new_30.csv",
#     "../embeddings/mixing_at_edge_bct_new_50.csv",
#     "../embeddings/mixing_at_edge_bct_new_70.csv",
#     "../embeddings/randomly_missing_bct_new_30.csv",
#     "../embeddings/randomly_missing_bct_new_50.csv",
#     "../embeddings/randomly_missing_bct_new_70.csv"
# ]

# # Initialize the list of embedding keys to use
# embedding_keys = ["X_scDREAMER", "X_ItClust"]
# file_paths = [
#     # "../embeddings/full_annotated_supervised_bct_new.csv",
#     # "../embeddings/randomly_missing_bct_new_30.csv",
#     # "../embeddings/randomly_missing_bct_new_50.csv",
#     # "../embeddings/randomly_missing_bct_new_70.csv"
#     "../embeddings/full_annotated_unsupervised_bct.csv"
# ]

# # Initialize the list of embedding keys to use
# # embedding_keys = ["X_scDREAMER", "X_ItClust"]
# embedding_keys = ["X_scanorama"]
file_paths = [
    "../embeddings/randomly_wrong_bct_30.csv",
    "../embeddings/randomly_wrong_bct_50.csv",
    "../embeddings/randomly_wrong_bct_70.csv"
]

# Initialize the list of embedding keys to use
embedding_keys = ["X_scANVI", "X_scgen", "X_stacas", "X_scDREAMER", "X_itclust"]

results_df = evaluate_embeddings(adata, file_paths, embedding_keys)

# Save the DataFrame to a CSV file
results_df.to_csv('../metrics/raw_results/bct_kbet_scores_randomly_wrong.csv', index=False)

File Path: ../embeddings/randomly_wrong_bct_30.csv, Embedding Key: X_scANVI, kBET Score: 0.21501850263241995
File Path: ../embeddings/randomly_wrong_bct_30.csv, Embedding Key: X_scgen, kBET Score: 0.06003446389536726
File Path: ../embeddings/randomly_wrong_bct_30.csv, Embedding Key: X_stacas, kBET Score: 0.08302349468652659
File Path: ../embeddings/randomly_wrong_bct_30.csv, Embedding Key: X_scDREAMER, kBET Score: 0.4611301853993022
Adding diffusion to step 4
Adding diffusion to step 5
Adding diffusion to step 6
Adding diffusion to step 7
Adding diffusion to step 8
Adding diffusion to step 9
Adding diffusion to step 4
Adding diffusion to step 5
Adding diffusion to step 4
Adding diffusion to step 5
Adding diffusion to step 6
Adding diffusion to step 7
File Path: ../embeddings/randomly_wrong_bct_30.csv, Embedding Key: X_itclust, kBET Score: 0.49968716655952694
File Path: ../embeddings/randomly_wrong_bct_50.csv, Embedding Key: X_scANVI, kBET Score: 0.21416678916433185
File Path: ../embedd

In [15]:
results_df

,file_path,embedding_key,kbet_score
0,../embeddings/randomly_wrong_bct_30.csv,X_scANVI,0.215019
1,../embeddings/randomly_wrong_bct_30.csv,X_scgen,0.060034
2,../embeddings/randomly_wrong_bct_30.csv,X_stacas,0.083023
3,../embeddings/randomly_wrong_bct_30.csv,X_scDREAMER,0.461130
4,../embeddings/randomly_wrong_bct_30.csv,X_itclust,0.499687
5,../embeddings/randomly_wrong_bct_50.csv,X_scANVI,0.214167
6,../embeddings/randomly_wrong_bct_50.csv,X_scgen,0.028796
7,../embeddings/randomly_wrong_bct_50.csv,X_stacas,0.093147
8,../embeddings/randomly_wrong_bct_50.csv,X_scDREAMER,0.449765
9,../embeddings/randomly_wrong_bct_50.csv,X_itclust,0.483411


## Print the overall metrics for LISI

In [19]:
# LISI
def calculate_LISI(adata, labels=None, total_cells=None, batch_key='batch', celltype_key='celltype', embed = 'X_pca'):
    if celltype_key is None:
        # Calculate bLISI
        lisi_b = 0
        lisi_c = lisi_f1 = np.nan
        if adata.shape[0] < 90:
            perplexity = int(adata.shape[0]/6)
        else:
            perplexity = 30
        lisi_res = lisi(adata.obsm[embed], adata.obs, batch_key, perplexity=perplexity)
        lisi_b = (np.mean(lisi_res)[batch_key]-1.)/(len(set(adata.obs[batch_key]))-1.)
    else:
        # Calculate 1-cLISI
        lisi_res = lisi(adata.obsm[embed], adata.obs, celltype_key)
        lisi_c = 1 - (np.mean(lisi_res)-1.)/(len(set(adata.obs[celltype_key]))-1.)

        # Calculate bLISI
        lisi_b = 0
        for label in labels:
            adata_sub = adata[adata.obs[celltype_key] == label]
            if adata_sub.shape[0] < 90:
                perplexity = int(adata_sub.shape[0]/6)
            else:
                perplexity = 30
            lisi_res = lisi(adata_sub.obsm[embed], adata_sub.obs, batch_key, perplexity=perplexity)
            lisi_batch = (np.mean(lisi_res)-1.)/(len(set(adata_sub.obs[batch_key]))-1.)
            lisi_b += lisi_batch*adata_sub.shape[0]
        lisi_b /= total_cells
        lisi_c = lisi_c.item()
        lisi_b = lisi_b.item()  # This will print only the value without the data type

        # Calcualte F1 score
        lisi_f1 = (2*lisi_c*lisi_b)/(lisi_c + lisi_b)
    
    return lisi_c, lisi_b, lisi_f1

In [20]:
def evaluate_embeddings(adata, file_paths, embedding_keys, batch_key='batch', celltype_key='cell_type'):
    results = []  # Initialize results as a list
    
    for file_path in file_paths:
        # Read the CSV file into a DataFrame
        combined_embeddings = pd.read_csv(file_path, index_col=0)
        
        # Ensure indices match
        combined_embeddings = combined_embeddings.loc[adata.obs_names]
        
        # Split the DataFrame into separate DataFrames for each embedding
        embeddings_dict = {key: combined_embeddings.filter(like=key) for key in embedding_keys}
        
        # Assign the embeddings back to adata.obsm
        for key, df in embeddings_dict.items():
            adata.obsm[key] = df.values
            
            # Evaluate embedding
            lisi_c, lisi_b, lisi_f1 = calculate_LISI(
                adata, labels=adata.obs[celltype_key].unique(), total_cells=adata.shape[0], batch_key=batch_key, celltype_key=celltype_key, embed=key
            )
            
            # Store results with identifying information
            result_entry = {
                'file_path': file_path,
                'embedding_key': key,
                'lisi_c': lisi_c,
                'lisi_b': lisi_b,
                'lisi_f1': lisi_f1
            }
            results.append(result_entry)
            
            # Print the file path, embedding key, and result
            print(f"File Path: {file_path}, Embedding Key: {key}, LISI_c: {lisi_c}, LISI_b: {lisi_b}, LISI_f1: {lisi_f1}")
    
    # Convert results list to DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df

In [21]:
file_paths = [
    "../embeddings/full_annotated_supervised_bct_new.csv",
    "../embeddings/randomly_missing_bct_new_30.csv",
    "../embeddings/randomly_missing_bct_new_50.csv",
    "../embeddings/randomly_missing_bct_new_70.csv"
]

# Initialize the list of embedding keys to use
embedding_keys = ["X_scDREAMER", "X_ItClust"]

results_df = evaluate_embeddings(adata, file_paths, embedding_keys)

# Save the DataFrame to a CSV file
results_df.to_csv('../metrics/bct_lisi_scores_new.csv', index=False)

File Path: ../embeddings/full_annotated_supervised_bct_new.csv, Embedding Key: X_scDREAMER, LISI_c: 0.9974974092878017, LISI_b: 0.34745860422772684, LISI_f1: 0.5153909184672552
File Path: ../embeddings/full_annotated_supervised_bct_new.csv, Embedding Key: X_ItClust, LISI_c: 0.9997575748055352, LISI_b: 0.3685781166533014, LISI_f1: 0.5385940984099244
File Path: ../embeddings/randomly_missing_bct_new_30.csv, Embedding Key: X_scDREAMER, LISI_c: 0.9959185127464507, LISI_b: 0.363749209620502, LISI_f1: 0.5328722097297502
File Path: ../embeddings/randomly_missing_bct_new_30.csv, Embedding Key: X_ItClust, LISI_c: 0.9588044165729593, LISI_b: 0.4799427409077123, LISI_f1: 0.6396832373107616
File Path: ../embeddings/randomly_missing_bct_new_50.csv, Embedding Key: X_scDREAMER, LISI_c: 0.9953478351399442, LISI_b: 0.3655866073107286, LISI_f1: 0.5347588051157426
File Path: ../embeddings/randomly_missing_bct_new_50.csv, Embedding Key: X_ItClust, LISI_c: 0.9100415162565337, LISI_b: 0.49372301925192374, L

In [22]:
results_df

,file_path,embedding_key,lisi_c,lisi_b,lisi_f1
0,../embeddings/full_annotated_supervised_bct_ne...,X_scDREAMER,0.997497,0.347459,0.515391
1,../embeddings/full_annotated_supervised_bct_ne...,X_ItClust,0.999758,0.368578,0.538594
2,../embeddings/randomly_missing_bct_new_30.csv,X_scDREAMER,0.995919,0.363749,0.532872
3,../embeddings/randomly_missing_bct_new_30.csv,X_ItClust,0.958804,0.479943,0.639683
4,../embeddings/randomly_missing_bct_new_50.csv,X_scDREAMER,0.995348,0.365587,0.534759
5,../embeddings/randomly_missing_bct_new_50.csv,X_ItClust,0.910042,0.493723,0.640148
6,../embeddings/randomly_missing_bct_new_70.csv,X_scDREAMER,0.994315,0.367914,0.537093
7,../embeddings/randomly_missing_bct_new_70.csv,X_ItClust,0.908284,0.466149,0.616102


# Combine the result for each embeddings

In [ ]:
import pandas as pd

# 读入Excel文件
file_path = './lung_atlas_metrics.csv'  # 替换为你的文件路径
df = pd.read_csv(file_path)

# 处理数据，对每一个setting生成一个表
settings = df['type'].unique()
output_file = pd.ExcelWriter('./lung_atlas_metrics_grouped.xlsx')

# 创建一个Excel writer对象，利用openpyxl引擎
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    for setting in settings:
        setting_df = df[df['type'] == setting]

        # 创建透视表，将行设为method，列设为metric，score为值
        pivot_table = setting_df.pivot(index='method', columns='metric', values='score')

        # 写入新的Excel sheet中
        pivot_table.to_excel(writer, sheet_name=setting)